In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from glob import glob
from tqdm import tqdm
from astropy.io import fits
import datetime
import time,sys
import astropy.units as u
import numpy as np
import matplotlib.pylab as plt
from tiles import Tile


In [2]:
avg_target_df=pd.read_csv('/Users/gstrampelli/PycharmProjects/Giovanni/work/database/FFP/targets_drc.csv')
avg_target_df

,target,file,filter,ra,dec,x,y,mag_aper,e_mag_aper,dist,snr
0,J155150.21-213457.4,iexn13010_drc.fits,F814W,237.959171,-21.582686,766.682062,870.519962,22.277353,0.021258,1.416799,227.932175
1,J164636.12-231337.6,iexn01010_drc.fits,F814W,251.650498,-23.227155,769.100793,866.146099,20.840983,0.005901,4.008029,792.416809
2,J164636.12-231337.6,iexn01020_drc.fits,F850LP,251.650498,-23.227156,768.870891,866.488792,19.749264,0.006932,3.617601,600.396423
3,J155416.68-263018.1,iexn02010_drc.fits,F814W,238.569423,-26.505131,762.415167,869.798303,21.448087,0.010977,5.588474,411.439362
4,J155416.68-263018.1,iexn02020_drc.fits,F850LP,238.569423,-26.505132,762.213142,870.180384,20.330281,0.012019,5.789669,338.973694
...,...,...,...,...,...,...,...,...,...,...,...
109,J042630.56+244355.5,iexn57020_drc.fits,F850LP,66.627360,24.732046,772.161171,860.967594,17.388929,0.000777,9.944833,5503.881348
110,J041548.09+291132.9,iexn59010_drc.fits,F814W,63.950387,29.192447,771.356066,869.879264,19.495704,0.001705,3.358237,2707.892578
111,J041548.09+291132.9,iexn59020_drc.fits,F850LP,63.950389,29.192447,771.125543,870.259044,18.268471,0.001755,3.136259,2408.281494
112,J042728.00+261204.7,iexn60010_drc.fits,F814W,66.866684,26.201263,770.474934,868.194087,19.229156,0.001359,3.063759,3397.833496


In [3]:
mvs_target_df=pd.read_csv('/Users/gstrampelli/PycharmProjects/Giovanni/work/database/FFP/targets_flc.csv')
mvs_target_df

,TARGET,file,x,y,dist,snr
0,J161754.11-232606.1,iexn12jvq_flc.fits,759.949707,821.036717,5.892971,82.648369
1,J164636.12-231337.6,iexn01oxq_flc.fits,760.166750,814.764820,5.816072,301.423645
2,J164636.12-231337.6,iexn01oyq_flc.fits,764.116000,816.179947,2.023375,282.925293
3,J164636.12-231337.6,iexn01ozq_flc.fits,762.529002,818.684984,2.564182,289.122620
4,J164636.12-231337.6,iexn01p0q_flc.fits,758.590255,817.187352,6.461054,284.427185
...,...,...,...,...,...,...
459,J042728.00+261204.7,iexn60olq_flc.fits,759.869457,819.409412,5.320613,1309.979614
460,J042728.00+261204.7,iexn60omq_flc.fits,761.124038,817.287748,3.940861,1232.124512
461,J042728.00+261204.7,iexn60onq_flc.fits,765.226449,818.745160,0.778809,1284.783569
462,J042728.00+261204.7,iexn60osq_flc.fits,763.572052,821.280073,3.577417,1178.333008


In [4]:
path2fits='/Users/gstrampelli/PycharmProjects/Giovanni/work/data/FFP/fits/'
for file in tqdm(glob('/Users/gstrampelli/PycharmProjects/Giovanni/work/data/FFP/catalogues/*.csv')):
    fitsname=file.split('/')[-1].split('.')[0]
    hdul=fits.open(path2fits+fitsname+'.fits')
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','filters']=hdul[0].header['FILTER']
    
    dateobs=hdul[0].header['DATE-OBS'].split('-')
    timeobs=hdul[0].header['TIME-OBS'].split(':')
    
    # df=pd.read_csv(file)
    date_time = datetime.datetime(int(dateobs[0]), int(dateobs[1]), int(dateobs[2]), int(timeobs[0]), int(timeobs[1]), int(timeobs[2]))
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','unittime']=time.mktime(date_time.timetuple())
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','fitsroot']=fitsname.split('_')[0]
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','filters']=hdul[0].header['FILTER']
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','ccd']=hdul[1].header['CCDCHIP']
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','visit']=str(fitsname[4:6])
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','exptime']=hdul[0].header['EXPTIME']
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','pav3']=hdul[0].header['PA_V3']
    mvs_target_df.loc[mvs_target_df.file==fitsname+'.fits','rota']=hdul[1].header['ORIENTAT']
 

100%|██████████| 412/412 [00:01<00:00, 363.88it/s]


In [5]:
mvs_target_df

,TARGET,file,x,y,dist,snr,filters,unittime,fitsroot,ccd,visit,exptime,pav3,rota
0,J161754.11-232606.1,iexn12jvq_flc.fits,759.949707,821.036717,5.892971,82.648369,F850LP,1.679281e+09,iexn12jvq,2.0,12,178.0,100.685303,145.215740
1,J164636.12-231337.6,iexn01oxq_flc.fits,760.166750,814.764820,5.816072,301.423645,F814W,1.678947e+09,iexn01oxq,2.0,01,178.0,97.360497,141.896045
2,J164636.12-231337.6,iexn01oyq_flc.fits,764.116000,816.179947,2.023375,282.925293,F814W,1.678948e+09,iexn01oyq,2.0,01,178.0,97.360527,141.896429
3,J164636.12-231337.6,iexn01ozq_flc.fits,762.529002,818.684984,2.564182,289.122620,F814W,1.678948e+09,iexn01ozq,2.0,01,178.0,97.360527,141.895127
4,J164636.12-231337.6,iexn01p0q_flc.fits,758.590255,817.187352,6.461054,284.427185,F814W,1.678948e+09,iexn01p0q,2.0,01,178.0,97.360497,141.896259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,J042728.00+261204.7,iexn60olq_flc.fits,759.869457,819.409412,5.320613,1309.979614,F814W,1.692223e+09,iexn60olq,2.0,60,178.0,82.269096,126.775930
460,J042728.00+261204.7,iexn60omq_flc.fits,761.124038,817.287748,3.940861,1232.124512,F850LP,1.692223e+09,iexn60omq,2.0,60,178.0,82.269096,126.782502
461,J042728.00+261204.7,iexn60onq_flc.fits,765.226449,818.745160,0.778809,1284.783569,F850LP,1.692223e+09,iexn60onq,2.0,60,178.0,82.269081,126.780277
462,J042728.00+261204.7,iexn60osq_flc.fits,763.572052,821.280073,3.577417,1178.333008,F850LP,1.692227e+09,iexn60osq,2.0,60,178.0,82.269073,126.782875


In [7]:
mvs_target_df.loc[mvs_target_df.filters=='F814W']

,TARGET,file,x,y,dist,snr,filters,unittime,fitsroot,ccd,visit,exptime,pav3,rota
1,J164636.12-231337.6,iexn01oxq_flc.fits,760.166750,814.764820,5.816072,301.423645,F814W,1.678947e+09,iexn01oxq,2.0,01,178.0,97.360497,141.896045
2,J164636.12-231337.6,iexn01oyq_flc.fits,764.116000,816.179947,2.023375,282.925293,F814W,1.678948e+09,iexn01oyq,2.0,01,178.0,97.360527,141.896429
3,J164636.12-231337.6,iexn01ozq_flc.fits,762.529002,818.684984,2.564182,289.122620,F814W,1.678948e+09,iexn01ozq,2.0,01,178.0,97.360527,141.895127
4,J164636.12-231337.6,iexn01p0q_flc.fits,758.590255,817.187352,6.461054,284.427185,F814W,1.678948e+09,iexn01p0q,2.0,01,178.0,97.360497,141.896259
9,J155416.68-263018.1,iexn02e3q_flc.fits,753.259138,818.650003,11.758842,156.033188,F814W,1.674520e+09,iexn02e3q,2.0,02,179.0,98.681168,143.217358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,J041548.09+291132.9,iexn59efq_flc.fits,760.651206,820.946803,5.253157,1019.242554,F814W,1.692057e+09,iexn59efq,2.0,59,178.0,82.072319,126.584019
456,J042728.00+261204.7,iexn60oiq_flc.fits,761.362280,816.845404,3.816556,1277.799561,F814W,1.692221e+09,iexn60oiq,2.0,60,178.0,82.269096,126.769092
457,J042728.00+261204.7,iexn60ojq_flc.fits,765.417200,818.406052,0.582180,1303.087524,F814W,1.692222e+09,iexn60ojq,2.0,60,178.0,82.269081,126.770620
458,J042728.00+261204.7,iexn60okq_flc.fits,763.841568,820.861524,3.087116,1275.769409,F814W,1.692222e+09,iexn60okq,2.0,60,178.0,82.269073,126.770873


In [8]:
mvs_target_df.loc[mvs_target_df.filters=='F850LP']

,TARGET,file,x,y,dist,snr,filters,unittime,fitsroot,ccd,visit,exptime,pav3,rota
0,J161754.11-232606.1,iexn12jvq_flc.fits,759.949707,821.036717,5.892971,82.648369,F850LP,1.679281e+09,iexn12jvq,2.0,12,178.0,100.685303,145.215740
5,J164636.12-231337.6,iexn01p1q_flc.fits,759.860639,815.166881,5.868526,225.692688,F850LP,1.678949e+09,iexn01p1q,2.0,01,178.0,97.360497,141.890553
6,J164636.12-231337.6,iexn01p2q_flc.fits,763.990396,816.515541,1.795249,207.028275,F850LP,1.678949e+09,iexn01p2q,2.0,01,178.0,97.360527,141.890804
7,J164636.12-231337.6,iexn01p3q_flc.fits,762.374175,819.081710,2.839904,220.992935,F850LP,1.678950e+09,iexn01p3q,2.0,01,178.0,97.360527,141.890573
8,J164636.12-231337.6,iexn01p4q_flc.fits,758.327324,817.628554,6.683007,212.522461,F850LP,1.678950e+09,iexn01p4q,2.0,01,178.0,97.360497,141.891097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,J041548.09+291132.9,iexn59ekq_flc.fits,760.417937,821.219986,5.600323,899.192871,F850LP,1.692062e+09,iexn59ekq,2.0,59,178.0,82.072319,126.581022
460,J042728.00+261204.7,iexn60omq_flc.fits,761.124038,817.287748,3.940861,1232.124512,F850LP,1.692223e+09,iexn60omq,2.0,60,178.0,82.269096,126.782502
461,J042728.00+261204.7,iexn60onq_flc.fits,765.226449,818.745160,0.778809,1284.783569,F850LP,1.692223e+09,iexn60onq,2.0,60,178.0,82.269081,126.780277
462,J042728.00+261204.7,iexn60osq_flc.fits,763.572052,821.280073,3.577417,1178.333008,F850LP,1.692227e+09,iexn60osq,2.0,60,178.0,82.269073,126.782875


In [9]:
mvs_target_df.loc[~mvs_target_df.x.isna()]

,TARGET,file,x,y,dist,snr,filters,unittime,fitsroot,ccd,visit,exptime,pav3,rota
0,J161754.11-232606.1,iexn12jvq_flc.fits,759.949707,821.036717,5.892971,82.648369,F850LP,1.679281e+09,iexn12jvq,2.0,12,178.0,100.685303,145.215740
1,J164636.12-231337.6,iexn01oxq_flc.fits,760.166750,814.764820,5.816072,301.423645,F814W,1.678947e+09,iexn01oxq,2.0,01,178.0,97.360497,141.896045
2,J164636.12-231337.6,iexn01oyq_flc.fits,764.116000,816.179947,2.023375,282.925293,F814W,1.678948e+09,iexn01oyq,2.0,01,178.0,97.360527,141.896429
3,J164636.12-231337.6,iexn01ozq_flc.fits,762.529002,818.684984,2.564182,289.122620,F814W,1.678948e+09,iexn01ozq,2.0,01,178.0,97.360527,141.895127
4,J164636.12-231337.6,iexn01p0q_flc.fits,758.590255,817.187352,6.461054,284.427185,F814W,1.678948e+09,iexn01p0q,2.0,01,178.0,97.360497,141.896259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,J042728.00+261204.7,iexn60olq_flc.fits,759.869457,819.409412,5.320613,1309.979614,F814W,1.692223e+09,iexn60olq,2.0,60,178.0,82.269096,126.775930
460,J042728.00+261204.7,iexn60omq_flc.fits,761.124038,817.287748,3.940861,1232.124512,F850LP,1.692223e+09,iexn60omq,2.0,60,178.0,82.269096,126.782502
461,J042728.00+261204.7,iexn60onq_flc.fits,765.226449,818.745160,0.778809,1284.783569,F850LP,1.692223e+09,iexn60onq,2.0,60,178.0,82.269081,126.780277
462,J042728.00+261204.7,iexn60osq_flc.fits,763.572052,821.280073,3.577417,1178.333008,F850LP,1.692227e+09,iexn60osq,2.0,60,178.0,82.269073,126.782875


In [10]:
elno=0
# mvs_target_df['avg_ids'] = np.nan
for TARGET in mvs_target_df.TARGET.unique():
    mvs_target_df.loc[mvs_target_df.TARGET == TARGET, 'avg_ids'] = int(elno)
    elno+=1

In [11]:
mvs_target_df = mvs_target_df.sort_values(['avg_ids','filters'])[['avg_ids', 'TARGET', 'file', 'x', 'y', 'dist', 'snr', 'filters','unittime','fitsroot','ccd','visit','exptime','pav3','rota']]

In [12]:
mvs_target_df.loc[~mvs_target_df.x.isna()&(mvs_target_df.filters=='F814W')]

,avg_ids,TARGET,file,x,y,dist,snr,filters,unittime,fitsroot,ccd,visit,exptime,pav3,rota
89,0.0,J161754.11-232606.1,iexn12jpq_flc.fits,757.574858,816.657408,7.545547,87.489143,F814W,1.679278e+09,iexn12jpq,2.0,12,178.0,100.685204,145.216510
90,0.0,J161754.11-232606.1,iexn12jqq_flc.fits,761.654031,818.049830,3.346340,91.702492,F814W,1.679279e+09,iexn12jqq,2.0,12,178.0,100.685303,145.217300
91,0.0,J161754.11-232606.1,iexn12jrq_flc.fits,760.134278,820.624555,5.528430,90.594002,F814W,1.679279e+09,iexn12jrq,2.0,12,178.0,100.685303,145.217636
92,0.0,J161754.11-232606.1,iexn12jsq_flc.fits,756.183371,819.160424,8.892668,87.450500,F814W,1.679279e+09,iexn12jsq,2.0,12,178.0,100.685204,145.217041
1,1.0,J164636.12-231337.6,iexn01oxq_flc.fits,760.166750,814.764820,5.816072,301.423645,F814W,1.678947e+09,iexn01oxq,2.0,01,178.0,97.360497,141.896045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,56.0,J041548.09+291132.9,iexn59efq_flc.fits,760.651206,820.946803,5.253157,1019.242554,F814W,1.692057e+09,iexn59efq,2.0,59,178.0,82.072319,126.584019
456,57.0,J042728.00+261204.7,iexn60oiq_flc.fits,761.362280,816.845404,3.816556,1277.799561,F814W,1.692221e+09,iexn60oiq,2.0,60,178.0,82.269096,126.769092
457,57.0,J042728.00+261204.7,iexn60ojq_flc.fits,765.417200,818.406052,0.582180,1303.087524,F814W,1.692222e+09,iexn60ojq,2.0,60,178.0,82.269081,126.770620
458,57.0,J042728.00+261204.7,iexn60okq_flc.fits,763.841568,820.861524,3.087116,1275.769409,F814W,1.692222e+09,iexn60okq,2.0,60,178.0,82.269073,126.770873


In [13]:
mvs_target_df.loc[~mvs_target_df.x.isna()&(mvs_target_df.filters=='F850LP')]

,avg_ids,TARGET,file,x,y,dist,snr,filters,unittime,fitsroot,ccd,visit,exptime,pav3,rota
0,0.0,J161754.11-232606.1,iexn12jvq_flc.fits,759.949707,821.036717,5.892971,82.648369,F850LP,1.679281e+09,iexn12jvq,2.0,12,178.0,100.685303,145.215740
93,0.0,J161754.11-232606.1,iexn12jtq_flc.fits,757.310277,817.015676,7.752466,73.934082,F850LP,1.679280e+09,iexn12jtq,2.0,12,178.0,100.685204,145.215840
94,0.0,J161754.11-232606.1,iexn12juq_flc.fits,761.477535,818.464391,3.552945,75.822121,F850LP,1.679280e+09,iexn12juq,2.0,12,178.0,100.685303,145.216088
95,0.0,J161754.11-232606.1,iexn12jwq_flc.fits,755.988511,819.732338,9.176488,67.658142,F850LP,1.679281e+09,iexn12jwq,2.0,12,178.0,100.685204,145.216598
5,1.0,J164636.12-231337.6,iexn01p1q_flc.fits,759.860639,815.166881,5.868526,225.692688,F850LP,1.678949e+09,iexn01p1q,2.0,01,178.0,97.360497,141.890553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,56.0,J041548.09+291132.9,iexn59ekq_flc.fits,760.417937,821.219986,5.600323,899.192871,F850LP,1.692062e+09,iexn59ekq,2.0,59,178.0,82.072319,126.581022
460,57.0,J042728.00+261204.7,iexn60omq_flc.fits,761.124038,817.287748,3.940861,1232.124512,F850LP,1.692223e+09,iexn60omq,2.0,60,178.0,82.269096,126.782502
461,57.0,J042728.00+261204.7,iexn60onq_flc.fits,765.226449,818.745160,0.778809,1284.783569,F850LP,1.692223e+09,iexn60onq,2.0,60,178.0,82.269081,126.780277
462,57.0,J042728.00+261204.7,iexn60osq_flc.fits,763.572052,821.280073,3.577417,1178.333008,F850LP,1.692227e+09,iexn60osq,2.0,60,178.0,82.269073,126.782875


In [14]:
for target in mvs_target_df.TARGET.unique():
    avg_target_df.loc[avg_target_df.target ==target,'avg_ids'] = mvs_target_df.loc[mvs_target_df.TARGET == target, 'avg_ids'].values[0]
avg_target_df

,target,file,filter,ra,dec,x,y,mag_aper,e_mag_aper,dist,snr,avg_ids
0,J155150.21-213457.4,iexn13010_drc.fits,F814W,237.959171,-21.582686,766.682062,870.519962,22.277353,0.021258,1.416799,227.932175,12.0
1,J164636.12-231337.6,iexn01010_drc.fits,F814W,251.650498,-23.227155,769.100793,866.146099,20.840983,0.005901,4.008029,792.416809,1.0
2,J164636.12-231337.6,iexn01020_drc.fits,F850LP,251.650498,-23.227156,768.870891,866.488792,19.749264,0.006932,3.617601,600.396423,1.0
3,J155416.68-263018.1,iexn02010_drc.fits,F814W,238.569423,-26.505131,762.415167,869.798303,21.448087,0.010977,5.588474,411.439362,2.0
4,J155416.68-263018.1,iexn02020_drc.fits,F850LP,238.569423,-26.505132,762.213142,870.180384,20.330281,0.012019,5.789669,338.973694,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
109,J042630.56+244355.5,iexn57020_drc.fits,F850LP,66.627360,24.732046,772.161171,860.967594,17.388929,0.000777,9.944833,5503.881348,55.0
110,J041548.09+291132.9,iexn59010_drc.fits,F814W,63.950387,29.192447,771.356066,869.879264,19.495704,0.001705,3.358237,2707.892578,56.0
111,J041548.09+291132.9,iexn59020_drc.fits,F850LP,63.950389,29.192447,771.125543,870.259044,18.268471,0.001755,3.136259,2408.281494,56.0
112,J042728.00+261204.7,iexn60010_drc.fits,F814W,66.866684,26.201263,770.474934,868.194087,19.229156,0.001359,3.063759,3397.833496,57.0


In [15]:
pos=0
mvs_dataframe=pd.DataFrame(columns=['avg_ids','ext', 'visit','x_f814w','y_f814w','x_f850lp','y_f850lp','fitsroot_f814w','fitsroot_f850lp', 'exptime_f814w','exptime_f850lp', 'pav3_f814w', 'pav3_f850lp', 'rota_f814w', 'rota_f850lp','flag_f814w','flag_f850lp'])
mvs_dataframe['flag_f814w']='rejected'
mvs_dataframe['flag_f850lp']='rejected'

for id in tqdm(mvs_target_df.avg_ids.unique()):
    df_F814W = mvs_target_df.loc[(mvs_target_df.avg_ids==id)&(mvs_target_df.filters=='F814W')].sort_values(['unittime'])
    df_F850LP = mvs_target_df.loc[(mvs_target_df.avg_ids==id)&(mvs_target_df.filters=='F850LP')].sort_values(['unittime'])
    num =  df_F814W.avg_ids.count() if df_F814W.avg_ids.count() >= df_F850LP.avg_ids.count() else df_F850LP.avg_ids.count()  
    for elno in range(num):
        mvs_dataframe.loc[pos,'avg_ids'] = id
        if not df_F814W.empty and elno <= df_F814W.avg_ids.count()-1:
            mvs_dataframe.loc[pos,'ext'] = 1 if df_F814W.ccd.unique() == 2 else 4
            mvs_dataframe.loc[pos,'visit'] = df_F814W.visit.unique()
            mvs_dataframe.loc[pos,['x_f814w','y_f814w','fitsroot_f814w','exptime_f814w','pav3_f814w','rota_f814w']] =df_F814W.iloc[elno][['x','y','fitsroot','exptime','pav3','rota']].values
        else:
            mvs_dataframe.loc[pos,'ext'] = 1 if df_F850LP.ccd.unique() == 2 else 4
            mvs_dataframe.loc[pos,'visit'] = df_F850LP.visit.unique()

        if not df_F850LP.empty and elno <= df_F850LP.avg_ids.count()-1:
            mvs_dataframe.loc[pos,['x_f850lp','y_f850lp','fitsroot_f850lp','exptime_f850lp','pav3_f850lp','rota_f850lp']] =df_F850LP.iloc[elno][['x','y','fitsroot','exptime','pav3','rota']].values
        pos+=1

mvs_dataframe

100%|██████████| 58/58 [00:00<00:00, 193.82it/s]


,avg_ids,ext,visit,x_f814w,y_f814w,x_f850lp,y_f850lp,fitsroot_f814w,fitsroot_f850lp,exptime_f814w,exptime_f850lp,pav3_f814w,pav3_f850lp,rota_f814w,rota_f850lp,flag_f814w,flag_f850lp
0,0.0,1,12,757.574858,816.657408,757.310277,817.015676,iexn12jpq,iexn12jtq,178.0,178.0,100.685204,100.685204,145.21651,145.21584,NaN,NaN
1,0.0,1,12,761.654031,818.04983,761.477535,818.464391,iexn12jqq,iexn12juq,178.0,178.0,100.685303,100.685303,145.2173,145.216088,NaN,NaN
2,0.0,1,12,760.134278,820.624555,759.949707,821.036717,iexn12jrq,iexn12jvq,178.0,178.0,100.685303,100.685303,145.217636,145.21574,NaN,NaN
3,0.0,1,12,756.183371,819.160424,755.988511,819.732338,iexn12jsq,iexn12jwq,178.0,178.0,100.685204,100.685204,145.217041,145.216598,NaN,NaN
4,1.0,1,01,760.16675,814.76482,759.860639,815.166881,iexn01oxq,iexn01p1q,178.0,178.0,97.360497,97.360497,141.896045,141.890553,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,56.0,1,59,760.651206,820.946803,760.417937,821.219986,iexn59efq,iexn59ekq,178.0,178.0,82.072319,82.072319,126.584019,126.581022,NaN,NaN
204,57.0,1,60,761.36228,816.845404,761.124038,817.287748,iexn60oiq,iexn60omq,178.0,178.0,82.269096,82.269096,126.769092,126.782502,NaN,NaN
205,57.0,1,60,765.4172,818.406052,765.226449,818.74516,iexn60ojq,iexn60onq,178.0,178.0,82.269081,82.269081,126.77062,126.780277,NaN,NaN
206,57.0,1,60,763.841568,820.861524,763.572052,821.280073,iexn60okq,iexn60osq,178.0,178.0,82.269073,82.269073,126.770873,126.782875,NaN,NaN


In [19]:
mvs_target_df.loc[mvs_target_df.avg_ids.isin(avg_target_df.loc[avg_target_df.target =='J042705.86+261520.3'].avg_ids.unique())]

,avg_ids,TARGET,file,x,y,dist,snr,filters,unittime,fitsroot,ccd,visit,exptime,pav3,rota
424,53.0,J042705.86+261520.3,iexn54clq_flc.fits,767.180470,807.572405,10.653130,138.444214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,53.0,J042705.86+261520.3,iexn54cmq_flc.fits,771.256534,809.021362,10.943498,153.308487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,53.0,J042705.86+261520.3,iexn54cnq_flc.fits,769.587923,811.441109,8.004255,149.754410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,53.0,J042705.86+261520.3,iexn54coq_flc.fits,765.380149,810.062673,7.946425,148.158844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,53.0,J042705.86+261520.3,iexn54cpq_flc.fits,766.576806,808.048239,10.075905,106.078011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,53.0,J042705.86+261520.3,iexn54cqq_flc.fits,770.592753,809.541668,10.140132,110.171074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
430,53.0,J042705.86+261520.3,iexn54crq_flc.fits,769.126971,811.944805,7.327842,110.900040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,53.0,J042705.86+261520.3,iexn54csq_flc.fits,765.073439,810.511900,7.488460,109.548782,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from utils_tile import small_tiles
path2tiles='/Users/gstrampelli/PycharmProjects/Giovanni/work/out/FFP/'
bad_F814W=[]#[124,154,178]
good_F814W=[]#[3,14,15,25,30,48,50,71,88,90,98,116,122,126,129,142,152,153,155,156,157,158,176,177,179,180,184,185,186,187,192,196]

bad_F850LP=[]#[153,157]
good_F850LP=[]#[23,26,27,36,39,59,92,97,118,120,127,144,152,155,159,168,169,170,171,178,182,184,185,192]
dict={'bad_F814W':bad_F814W,'bad_F850LP':bad_F850LP,'good_F814W':good_F814W,'good_F850LP':good_F850LP}


mvs_dataframe = small_tiles(mvs_dataframe,path2fits, path2tiles, ['F814W','F850LP'], dict=dict,nrows=21, ncols=10)

In [ ]:
mvs_dataframe.loc[mvs_dataframe['flag_f814w'].isna(),'flag_f814w'] = 'rejected'
mvs_dataframe.loc[mvs_dataframe['flag_f850lp'].isna(),'flag_f850lp'] = 'rejected'


In [ ]:
mvs_dataframe = mvs_dataframe.reset_index().rename(columns={'index':'mvs_ids'})
mvs_dataframe['avg_ids']=mvs_dataframe.avg_ids.astype(int)
mvs_dataframe


In [ ]:
mvs_dataframe.to_csv('/Users/gstrampelli/PycharmProjects/Giovanni/work/database/FFP/FFP_mvs_target_photometry.csv')

In [ ]:
pos=0
# avg_target_df = avg_target_df[['avg_ids','ra','dec','mag_psf','e_mag_psf']].sort_values(by='avg_ids').reset_index(drop=True)
avg_dataframe=pd.DataFrame(columns=['avg_ids','ra', 'dec','m_f814w','e_f814w','m_f850lp','e_f850lp','type'])

for id in tqdm(avg_target_df.avg_ids.unique()):
    df_F814W = avg_target_df.loc[(avg_target_df.avg_ids==id)&(avg_target_df['filter']=='F814W')]
    df_F850LP = avg_target_df.loc[(avg_target_df.avg_ids==id)&(avg_target_df['filter']=='F850LP')]
    num =  df_F814W.avg_ids.count() if df_F814W.avg_ids.count() >= df_F850LP.avg_ids.count() else df_F850LP.avg_ids.count()  
    for elno in range(num):
        avg_dataframe.loc[pos,'avg_ids'] = id
        if not df_F814W.empty:
            avg_dataframe.loc[pos,'ra'] = df_F814W.ra.unique() 
            avg_dataframe.loc[pos,'dec'] = df_F814W.dec.unique()
        elif not df_F850LP.empty:
            avg_dataframe.loc[pos,'ra'] = df_F850LP.ra.unique() 
            avg_dataframe.loc[pos,'dec'] = df_F850LP.dec.unique()
        avg_dataframe.loc[pos,'type'] = 1
        if np.any(mvs_dataframe.loc[mvs_dataframe.avg_ids==id,['flag_f814w','flag_f850lp']]=='bad_target'):
                avg_dataframe.loc[avg_dataframe.avg_ids==id,'type']=0
        if not df_F814W.empty and elno <= df_F814W.avg_ids.count()-1:
            avg_dataframe.loc[pos,['m_f814w','e_f814w']] =df_F814W.iloc[elno][['mag_psf','e_mag_psf']].values

        if not df_F850LP.empty and elno <= df_F850LP.avg_ids.count()-1:
            avg_dataframe.loc[pos,['m_f850lp','e_f850lp']] =df_F850LP.iloc[elno][['mag_psf','e_mag_psf']].values
        pos+=1
avg_dataframe['avg_ids']=avg_dataframe.avg_ids.astype(int)
avg_dataframe

In [ ]:
avg_dataframe.to_csv('/Users/gstrampelli/PycharmProjects/Giovanni/work/database/FFP/FFP_avg_target_photometry.csv')

In [ ]:
avg_target_df